In [ ]:
%pip install GEOparse
%pip install pysradb

In [ ]:
import pysradb
import pandas as pd

In [ ]:
srp_list = [
    "SRP057214","SRP067879","SRP076092","SRP095521","SRP103822",
    "SRP104427","SRP132878","SRP128509","SRP131887","SRP164773",
    "SRP153920","SRP199944","SRP201872","SRP225676","SRP251750",
    "SRP286420","SRP255727","SRP250664","SRP324876","SRP330518",
    "SRP339135","SRP359061","SRP374794","SRP250379","SRP413301",
    "SRP378818","SRP124850","SRP316044","SRP201609","SRP359061"
    ]

In [ ]:
bioproject = [
    "PRJNA281260","PRJNA307178","PRJNA324370","PRJNA358485","PRJNA382636",
    "PRJNA383707","PRJNA434118","PRJNA429185","PRJNA432384","PRJNA495394",
    "PRJNA481337","PRJNA545540","PRJNA549666","PRJNA577587","PRJNA610625",
    "PRJNA667476","PRJNA623860","PRJNA608652","PRJNA739695","PRJNA750873",
    "PRJNA767051","PRJNA804872","PRJNA837255","PRJNA608124","PRJNA912722",
    "PRJNA844235","PRJNA418134","PRJNA724680","PRJNA549258","PRJNA804872"    
]

In [ ]:
pubmed = [
    "25934506","27036006","27688757","28167766","28607059",
    "28607063","29581276","29666379","29735688","30413533",
    "30659176","31217280","31217280","31624146","32299912",
    "33087442","33771559","34362292","34375638","34525225",
    "35217627","35339490","35994676","36155646","36775124",
    "36715337","29282293","34071702","32355176","35339490"  
]

In [ ]:
import subprocess

gse_list = []
for srp in srp_list:
    gse = subprocess.check_output([f"pysradb srp-to-gse {srp}"], shell=True)
    gse_list.append(str(gse.strip().split()[-1]).split("'")[1].replace("{}","NA"))

In [ ]:
srp_geo_df = pd.DataFrame({'study_accession':srp_list, 'geo_accession':gse_list})

In [ ]:
srp_df_all = pd.DataFrame()
for srp in srp_list:
    srp_df = pysradb.SRAweb().sra_metadata(srp)
    srp_df_all = pd.concat([srp_df_all, srp_df], ignore_index = True)

In [ ]:
drop_list = ["sample_title","experiment_desc","library_name","instrument_model","instrument_model_desc"]

In [ ]:
srp_df_all_filt = srp_df_all.loc[srp_df_all.library_strategy.isin(['OTHER', 'WGS'])].drop(columns=drop_list).loc[~srp_df_all.experiment_title.str.contains('PADD-seq|EdU-seq')]

In [ ]:
srp_df_all_filt = pd.merge(srp_df_all_filt, srp_geo_df, how='left', 
    on=['study_accession'])

In [ ]:
srp_df_ordered = srp_df_all_filt.reindex(columns=[
    'study_accession', 'geo_accession', 'experiment_accession', 
    'sample_accession', 'run_accession', 'experiment_title', 
    'organism_taxid', 'organism_name', 'library_strategy', 'library_source', 
    'library_selection', 'library_layout', 'instrument', 'total_spots',
    'total_size', 'run_total_spots', 'run_total_bases'
    ])

In [ ]:
srp_df_all.to_csv("info.csv")

In [ ]:
df = pysradb.SRAweb().search_sra(search_str="xr-seq|damage-seq")

In [ ]:
import inspect
lines = inspect.getsource(pysradb)

print(lines)

Practice other utilities

In [ ]:
import GEOparse

with open("info2.txt", "w") as info:
    gse = GEOparse.get_GEO(geo="GSE67941", destdir="./")

    info.write("GSM example:\n")
    for gsm_name, gsm in gse.gsms.items():
        info.write(f"Name: {gsm_name}\n")
        info.write("Metadata:\n")
        for key, value in gsm.metadata.items():
            for val in value:
                info.write(f"{value}\n")

    info.write("GPL example:\n")
    for gpl_name, gpl in gse.gpls.items():
        info.write(f"Name: {gpl_name}\n")
        info.write("Metadata:\n")
        for key, value in gpl.metadata.items():
            info.write(" - %s : %s\n" % (key, ", ".join(value)))

In [ ]:
gse = GEOparse.get_GEO(geo="GSE67941", destdir="./")

gse.metadata.values()

In [ ]:
!pysradb metadata SRP104427 --detailed > info3.txt

In [ ]:
!pysradb metadata SRP104427 --detailed | awk '{print $1","$2}'

In [ ]:
!pysradb srp-to-gse SRP104427

In [ ]:
db = pysradb.SRAweb()
df = db.sra_metadata('SRP104427')
df.head()

In [ ]:
df_ds = db.search_sra(search_str="damage-seq")
len(df_ds["study_title"].unique())

In [ ]:
df_ds["study_title"].unique()

In [ ]:
df_xr = db.search_sra(search_str="XR-seq")
len(df_xr["study_title"].unique())

In [ ]:
df_xr["study_title"].unique()

In [ ]:
df_filt = df[["study_accession", "study_title"]]
df_filt.drop_duplicates()